In [6]:
from tkinter import *
from tkinter import ttk
import pymysql

# Create the main window
root = Tk()
root.title("Simple Tkinter App")

# Function to save form data to the database
def save_data():
    conn = pymysql.connect(
       host='localhost', user='root', password='123456', db='form'
    )
    cursor = conn.cursor()
    
    # Create table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS application (
        serialno INT,
        firstname VARCHAR(255),
        lastname VARCHAR(255),
        fathersname VARCHAR(255),
        mothersname VARCHAR(255),
        address VARCHAR(255),
        sex VARCHAR(10),
        qualification VARCHAR(255)
    )
    ''')

    # Determine the selected sex
    sex = 'Male' if male_var.get() else 'Female' if female_var.get() else ''
    
    # Insert form data into the table
    cursor.execute('''
    INSERT INTO application (serialno, firstname, lastname, fathersname, mothersname, address, sex, qualification)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        serial_no_entry.get(),
        e1.get(),
        e2.get(),
        e3.get(),
        e4.get(),
        e5.get(),
        sex,
        qualification_var.get()
    ))
    
    conn.commit()
    conn.close()

    # Optionally, you can clear the form fields after submission
    serial_no_entry.delete(0, END)
    e1.delete(0, END)
    e2.delete(0, END)
    e3.delete(0, END)
    e4.delete(0, END)
    e5.delete(0, END)
    male_var.set(0)
    female_var.set(0)
    qualification_var.set("Select")

# Function to view stored records
def view_records(serial_no=None, name=None):
    conn = pymysql.connect(
        host='localhost',
        user='root',
        password='123456',
        database='form'
    )
    cursor = conn.cursor()
    
    if serial_no:
        cursor.execute('SELECT * FROM application WHERE serialno = %s', (serial_no,))
    elif name:
        cursor.execute('SELECT * FROM application WHERE firstname LIKE %s OR lastname LIKE %s', (f'%{name}%', f'%{name}%'))
    else:
        cursor.execute('SELECT * FROM application')
    
    records = cursor.fetchall()
    conn.close()
    
    # Create a new window to display the records
    records_window = Toplevel(root)
    records_window.title("View Records")
    
    tree = ttk.Treeview(records_window, columns=("serialno", "firstname", "lastname", "fathersname", "mothersname", "address", "sex", "qualification"), show='headings')
    
    tree.heading("serialno", text="Serial No")
    tree.heading("firstname", text="First Name")
    tree.heading("lastname", text="Last Name")
    tree.heading("fathersname", text="Father's Name")
    tree.heading("mothersname", text="Mother's Name")
    tree.heading("address", text="Address")
    tree.heading("sex", text="Sex")
    tree.heading("qualification", text="Qualification")
    
    for record in records:
        tree.insert("", END, values=record)
    
    tree.pack(fill=BOTH, expand=True)

# Function to handle the search button click
def search_record():
    serial_no = search_entry.get()
    name = name_search_entry.get()
    if serial_no:
        view_records(serial_no=serial_no)
    elif name:
        view_records(name=name)
    else:
        view_records()

# Create a label and textbox for Serial No at the top of the window
Label(root, text='Serial No').grid(row=0, column=0, padx=10, pady=5, sticky=E)
serial_no_entry = Entry(root)
serial_no_entry.grid(row=0, column=1, padx=10, pady=5)

# Create a label with text and use grid for layout
w = Label(root, text='GeeksForGeeks.org!')
w.grid(row=1, column=0, columnspan=2, pady=10)  # Using grid and spanning two columns

# Create labels for First Name, Last Name, Father's Name, Mother's Name, and Address
Label(root, text='First Name').grid(row=2, column=0, padx=10, pady=5, sticky=E)
Label(root, text='Last Name').grid(row=3, column=0, padx=10, pady=5, sticky=E)
Label(root, text="Father's Name").grid(row=4, column=0, padx=10, pady=5, sticky=E)
Label(root, text="Mother's Name").grid(row=5, column=0, padx=10, pady=5, sticky=E)
Label(root, text='Address').grid(row=6, column=0, padx=10, pady=5, sticky=E)

# Create entry widgets for First Name, Last Name, Father's Name, Mother's Name, and Address
e1 = Entry(root)
e2 = Entry(root)
e3 = Entry(root)
e4 = Entry(root)
e5 = Entry(root)

# Arrange the entry widgets using grid geometry manager
e1.grid(row=2, column=1, padx=10, pady=5)
e2.grid(row=3, column=1, padx=10, pady=5)
e3.grid(row=4, column=1, padx=10, pady=5)
e4.grid(row=5, column=1, padx=10, pady=5)
e5.grid(row=6, column=1, padx=10, pady=5)

# Create a label for the sex section
Label(root, text='Sex').grid(row=7, column=0, padx=10, pady=5, sticky=E)

# Variables to hold the state of the checkbuttons
male_var = IntVar()
female_var = IntVar()

# Create checkbuttons for male and female
male_check = Checkbutton(root, text='Male', variable=male_var)
female_check = Checkbutton(root, text='Female', variable=female_var)

# Arrange the checkbuttons using grid geometry manager
male_check.grid(row=7, column=1, padx=10, pady=5, sticky=W)
female_check.grid(row=8, column=1, padx=10, pady=5, sticky=W)

# Create a label for the qualification section
Label(root, text='Qualification').grid(row=9, column=0, padx=10, pady=5, sticky=E)

# Variable to hold the selected qualification
qualification_var = StringVar(root)
qualification_var.set("Select")  # Set the default value

# Create an option menu for qualifications
qualifications = ["B.E", "M.E", "B.A", "M.A", "MBA", "B.Sc", "MBBS", "BDS"]
qualification_menu = OptionMenu(root, qualification_var, *qualifications)

# Arrange the option menu using grid geometry manager
qualification_menu.grid(row=9, column=1, padx=10, pady=5, sticky=W)

# Create the Submit button
submit_button = Button(root, text="Submit", command=save_data)
submit_button.grid(row=10, column=0, columnspan=2, pady=10)

# Create a label and textbox for Search Serial No
Label(root, text='Search Serial No').grid(row=12, column=0, padx=10, pady=5, sticky=E)
search_entry = Entry(root)
search_entry.grid(row=12, column=1, padx=10, pady=5)

# Create the Search button
search_button = Button(root, text="Search", command=search_record)
search_button.grid(row=13, column=0, columnspan=2, pady=10)

# Create a label and textbox for Search by Name
Label(root, text='Search by Name').grid(row=14, column=0, padx=10, pady=5, sticky=E)
name_search_entry = Entry(root)
name_search_entry.grid(row=14, column=1, padx=10, pady=5)

# Create the Search by Name button
name_search_button = Button(root, text="Search by Name", command=search_record)
name_search_button.grid(row=15, column=0, columnspan=2, pady=10)

# Create the View Records button
view_button = Button(root, text="View All Records", command=view_records)
view_button.grid(row=16, column=0, columnspan=2, pady=10)

# Run the main event loop
root.mainloop()
